In [11]:
import getpass
import os
import cohere
from langchain_cohere import ChatCohere

os.environ["COHERE_API_KEY"] = getpass.getpass("Enter your Cohere API Key: ")
# uVyZSCrGIWJcrJxEAqTuHirWOEazjbgXlJMlLPm1
# co = cohere.Client(os.environ["COHERE_API_KEY"])
model = ChatCohere(model="command-r-08-2024", safety_mode="CONTEXTUAL")

In [12]:
from langchain_core.output_parsers import StrOutputParser
from prompt import topics, javanese_overgeneration_prompt, sundanese_overgeneration_prompt

# Function to create a chain for the selected language
def create_chain(prompt_template, model):
    return prompt_template | model | StrOutputParser()

chain_sundanese = create_chain(sundanese_overgeneration_prompt, model)
chain_javanese = create_chain(javanese_overgeneration_prompt, model)

In [13]:
chain_javanese

FewShotPromptTemplate(input_variables=['topic'], example_selector=<prompt.CustomExampleSelector object at 0x105913ee0>, example_prompt=PromptTemplate(input_variables=['correct_ending', 'story_premise', 'wrong_ending'], template='Story Premise: {story_premise}\nCorrect Ending: {correct_ending}\nIncorrect Ending: {wrong_ending}'), suffix='Please generate several triplets, strictly following the format in the examples, do not add bullets or any additional response.', prefix='Your task is to write several triplets of story premises consisting of four sentences, wrong ending, and correct ending in Javanese. Include Javanese cultural values in the story with the topic "{topic}". Here are some examples of the story format:')
| ChatCohere(client=<cohere.client.Client object at 0x13cc53be0>, async_client=<cohere.client.AsyncClient object at 0x13cc53f10>, model='command-r-08-2024', cohere_api_key=SecretStr('**********'))
| StrOutputParser()

In [14]:
def generate_examples(language_chain, topic, n, batch_size=5):
    num_batches = (n + batch_size - 1) // batch_size 
    examples = []
    
    for _ in range(num_batches):
        batch_input = [{"topic": topic}] * batch_size
        batch_output = language_chain.batch(batch_input)
        examples.extend(batch_output)
    
    return examples[:n] 

In [15]:
def generate_examples_javanese(topic, n):
    return generate_examples(chain_javanese, topic, n)

def generate_examples_sundanese(topic, n):
    return generate_examples(chain_sundanese, topic, n)

In [16]:
selected_topic = topics[0]
print(selected_topic)

javanese_examples = generate_examples_javanese(topic = selected_topic, n=1)

Food (e.g.: food souvenir, traditional foods and beverages, eating habit, traditional cutlery or cooking ware, local fruit)


In [17]:
print(javanese_examples[0])

Story Premise: Ngluwihake budaya kulinere, Pak Lik ngrencanake tur marang museum kuliner. Museum iki nggawe kagetan, nggawe wong kaget karo koleksi sing ajaib. Pak Lik nggawe tur iki kanggo ngajake wong-wong marang budaya kulinere.
Correct Ending: Tur iki nggawe wong kaget, tur nggawe wong kangen marang rasa-rasa sing langka.
Incorrect Ending: Tur iki ora sukses, wong-wong ora tertarik marang budaya kulinere.

Story Premise: Mbok Darso nggawe kerupuk sing ajaib, ora nggunakake bahan-bahan sing biasa. Kerupuk iki ora biasa, ora nggunakake bahan-bahan sing umum. Mbok Darso nggawe kerupuk iki kanggo nggawe wong penasaran.
Correct Ending: Kerupuk iki nggawe wong penasaran, lan wong-wong nggawe antusias marang rasa sing anyar.
Incorrect Ending: Kerupuk iki ora digemari, wong-wong ora suka marang rasa sing anyar.

Story Premise: Pak Lik ngrencanake acara nguri-uri budaya kulinere, nggawe acara nggawe wong penasaran marang rasa-rasa sing langka. Acara iki nggawe wong kaget, nggawe wong kangen

In [9]:
javanese_examples

['Story Premise: "Kapan saiki nggih, Mas? Aku arep mangan sing endhog, nanging aku ora duwe duit." Pitutur bocah wadon marang penjual sing nunggu ing warung tenda. "Aku arep ngutang, nanging aku ora tau ngutang, aku ora duwe duit." "Ya wis, nggih, aku ngerti, aku ora bakal ngutangke, nanging aku arep nggawe sing endhog, aku arep nggawe sing endhog kanggo kowe."\nCorrect Ending: Bocah wadon iku ora duwe duit, nanging penjual iku ora ngutangke, nanging nggawe sing endhog kanggo kersane.\nIncorrect Ending: Penjual iku ngutangke bocah wadon iku, nanging bocah wadon iku ora duwe duit kanggo ngbayar.\n\nStory Premise: "Aku arep mangan sing endhog, nanging aku ora duwe duit." Pitutur bocah wadon marang penjual sing nunggu ing warung tenda. "Ya wis, nggih, aku ngerti, aku ora bakal ngutangke, nanging aku arep nggawe sing endhog kanggo kowe." "Aku ora duwe duit, nanging aku arep mangan sing endhog."\nCorrect Ending: Penjual iku nggawe sing endhog kanggo bocah wadon iku, nanging bocah wadon iku 